In [1]:
from IPython.core import display
from io import BytesIO
from PIL import Image
import numpy as np
import random
import os
import neurolab as nl
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import time
import math
import pylab as pl

%matplotlib inline

In [2]:
os.chdir("..")
%pwd

u'C:\\Users\\joeal_000\\Documents\\GitHub\\compOpt_Project'

In [3]:
def getData(path):
    "Splits the data into X and y both being numpy arrays (data already normalized)"
    f = open(path)
    lines = f.readlines()
    f.close
    
    images = []
    classification = []
    for line in lines:
        nums = line.split()
        classification.append(nums[0])
        images.append([((float (val))) for val in nums[1:]])
    return (np.array(classification, dtype=np.float),np.array(images))

def display_grayscale(arr):
    "SIDE EFFECTS: INTENDED TO BE USED IN IPYTHON NOTEBOOK"
    img = np.array([round(val,2) for val in arr]).astype('uint8').reshape((16,16))
    plt.imshow(img, cmap = cm.Greys_r)
    return Image.fromarray(img)

In [4]:
classification = np.loadtxt('classification.txt')
data = np.loadtxt('features/data80NetFeatures.txt')

In [5]:
def thisThat(x):
    if x == 1:
        return 1
    else:
        return -1

In [6]:
training = data[:300]
trainyans = map(thisThat,classification[:300])
Dtest = data[300:]
testyans = map(thisThat,classification[300:])

In [33]:

def makeWeights(init, layers):
    weights = [0]
    
    if init == "rand":
        for i in range(len(layers) - 1):
            if i == len(layers) - 2:
                weights.append(np.array([random.uniform(0, 1) for x in range((layers[i] * layers[i + 1]))]).reshape(layers[i], layers[i + 1]))
            else:
                weights.append(np.array([random.uniform(0, 1) for x in range((layers[i] * (layers[i + 1] - 1)))]).reshape(layers[i], (layers[i + 1] - 1)))
        return weights
    
    else:
        for i in range(len(layers) - 1):
            if i == len(layers) - 2:
                weights.append(np.array([init] * (layers[i] * layers[i + 1])).reshape(layers[i], layers[i + 1]))
            else:
                weights.append(np.array([init] * (layers[i] * (layers[i + 1] - 1))).reshape(layers[i], (layers[i + 1] - 1)))
        return weights
    
            
def forwardProp(x0, weights):
    xl = [np.array(x0)]
    for l in range(1,len(weights)):
        sl = weights[l].T.dot(xl[-1])
        xl.append(np.append(1,np.tanh(sl)))
        
    xl[-1] = xl[-1][1:][0]
    return xl

def classify(x0, weights):
    xl = [np.array(x0)]
    for l in range(1,len(weights)):
        sl = weights[l].T.dot(xl[-1])
        xl.append(np.append(1,np.tanh(sl)))
    return np.sign(xl[-1][1])
    

def backProp(xl, weights):
    dl = [np.array(1 - xl[-1] ** 2)]
    for l in reversed(range(1, len(weights) - 1)):
        diag = 1 - xl[l] ** 2
        tl = np.diagflat(diag[1:])
        
        product = weights[l + 1].dot(dl[0])
        dl[0:0] = [tl.dot(product[1:])]
    dl [0:0] = [0]
    return dl


def gradient(data, y, layers, weights):
    
    Ein = 0
    gl = [0] * len(weights)
    
    for i in range(1,len(data)):
        x0 = data[i]
        xl = forwardProp(x0, weights)
        for x in range(len(xl) - 1):
            xl[x] = np.array([[d] for d in xl[x]])
        dl = backProp(xl, weights)
        Ein += (1.0 / (4 * (len(data) - 1))) * ((xl[-1] - y[i]) ** 2)

        for l in range(1,len(weights)):
            glxn = (2 * (xl[-1] - y[i])) * (xl[l - 1].dot(dl[l].T)) #+ (0.01 / len(data))
            gl[l] = gl[l] + (1.0 / (len(data) - 1)) * (glxn / (len(data) - 1))
            
    return [gl, Ein]

def stochastic(data, y, layers, weights):
    
    alleins = []
    minChange = 1
    minEin = .0000000000000000000005
    maxIter = 2
    learningRate = .01
    
    Es = []
    
    iterr = 0
    Ein = 0
    change = 10
    
    #for i in range(1, len(data)):
        #data[i][0:0] = [1]
    
    while (iterr < maxIter) and (not((change <= minChange) and (Ein <= minEin))):
        ind = random.randint(1,len(data)) - 1
        point = [data[ind]]
        ans = [y[ind]]
        point[0:0] = [0]
        ans[0:0] = [0]
        gl,Ein = gradient(point, ans, layers, weights)
        alleins.append(Ein)
            
        change =  0
        
        print classify(data[ind],weights)
        print ans[1]
        for l in range(1, len(weights)):
            temp = np.sum(weights[l])
            weights[l] -= learningRate * gl[l]
            change += abs(temp - np.sum(weights[l]))
            
        iterr += 1
    print iterr
    return [weights,alleins]
    
    print iterr
    return [weights,alleins]
#print stochastic([0, [1, 1]], [0, 1], layers, makeWeights(.25, layers))

In [34]:
layers = [80, 10, 1]
weights,eins = stochastic(training, trainyans, layers, makeWeights("rand", layers))

1.0
-1
-1.0
-1
2


In [18]:
def Etest(weights, test, testyans):
    
    count = 0.0
    
    for i in range(len(test)):
        n = [1] + test[i]
 
        
        summ = 0
        if (classify(n, weights) == 1) and (testyans[i] == -1):
            count += 1.0
            
        if (classify(n, weights) == -1) and (testyans[i] == 1):
            count += 1.0
                   
    return count / len(test)


print Etest(weights, Dtest, testyans)
print 1 - Etest(weights, Dtest, testyans)

NameError: name 'weights' is not defined

[  1.22124533e-15   1.83323395e-03   4.26149116e-11   2.00000000e+00
   1.21029782e-05   1.99999719e+00   1.71428742e-04   2.00000000e+00
   2.90595032e-07   2.00000000e+00   4.92185053e-03   1.11886711e+00
   9.24155019e-04   1.99893351e+00   4.17691477e-01   6.24644054e-04
   6.72032167e-02   2.00000000e+00   9.54151298e-01   1.99953142e+00
   6.02085359e-01   1.99992497e+00   2.00000000e+00   1.25993576e-04
   3.03333630e-05   1.99999186e+00   3.49937312e-09   6.26129623e-01
   2.00000000e+00   5.04920661e-10   2.00000000e+00   1.99999999e+00
   1.74125006e-05   2.00000000e+00   1.99947184e+00   1.80002070e+00
   1.85703626e-06   1.35448588e+00   1.67298525e-04   5.90360065e-06
   2.80114608e-04   0.00000000e+00   1.99950700e+00   1.96495249e+00
   1.99974367e+00   1.40003614e+00   1.11286766e-05   2.00000000e+00
   1.99999984e+00   1.99999823e+00   0.00000000e+00   2.00000000e+00
   1.99993236e+00   2.00000000e+00   1.55250477e-03   1.99997198e+00
   3.01088510e-09   1.99999459e+00

In [25]:
trainyans[1]

-1